In [1]:
import pandas as pd

data = pd.read_csv("dataset/mta-data/MTA_Turnstile_2014_2.csv", sep=',', header=None, usecols=[1,6,7,9,10])
data.columns = ["Remote", "Date", "Hour", "Entry", "Exit"]

In [2]:
remote_data = pd.read_csv("dataset/Remote-Booth-Station.csv", sep=',', usecols=[0,2])
remote_data = remote_data.drop_duplicates()
remote_data = remote_data.drop_duplicates(['Remote'], keep='first')

In [3]:
merged = data.merge(remote_data,how='inner',on='Remote')

In [4]:
weather_data = pd.read_csv("dataset/clean-weather-data/weather.csv", sep=',', header=None)
weather_data.columns = ['Date','Event']
weather_data = weather_data.fillna('Normal')

In [27]:
remerged = merged.merge(weather_data,how='inner',on='Date')
remerged = remerged.set_index(['Remote','Date','Hour','Station','Event']) \
                    .diff() \
                    .fillna(float(0)) \
                    .reset_index()
remerged.loc[(remerged.Entry < 0.0) | (remerged.Entry > 10000.0), ['Entry','Exit']] = float(0)
remerged.head(50)

,Remote,Date,Hour,Station,Event,Entry,Exit
0,R001,10/11/2014,01:00:00,WHITEHALL ST,Rain,0.0,0.0
1,R001,10/11/2014,05:00:00,WHITEHALL ST,Rain,20.0,6.0
2,R001,10/11/2014,09:00:00,WHITEHALL ST,Rain,130.0,11.0
3,R001,10/11/2014,13:00:00,WHITEHALL ST,Rain,245.0,26.0
4,R001,10/11/2014,17:00:00,WHITEHALL ST,Rain,316.0,39.0
5,R001,10/11/2014,21:00:00,WHITEHALL ST,Rain,281.0,40.0
6,R001,10/11/2014,01:00:00,WHITEHALL ST,Rain,0.0,0.0
7,R001,10/11/2014,05:00:00,WHITEHALL ST,Rain,9.0,2.0
8,R001,10/11/2014,09:00:00,WHITEHALL ST,Rain,109.0,4.0
9,R001,10/11/2014,13:00:00,WHITEHALL ST,Rain,179.0,13.0
